In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import time
from scipy.special import jv

In [2]:
from nomaenv import NomaEnv
from algorithms.baselines import RandomScheduler, EarliestDeadlineFirstScheduler, SlottedAloha

In [3]:
k = 4
deadlines = np.array([5 for _ in range(k)])
offsets = np.zeros(k)
arrival_probs = np.array([0.8 for _ in range(k)])
period = 5

env = NomaEnv(k,
              deadlines,
              lbda=1,
              period=period,
              arrival_probs=arrival_probs,
              offsets=offsets,
              episode_length=10,
              max_simultaneous_devices=3,
              traffic_model='aperiodic',
              channel_model='interference',
              distances=65,
              path_loss=True,
              shadowing=False,
              fast_fading=True,
              verbose=True
             )

Distances: [65. 65. 65. 65.]
Gain UE: [-90.70970328 -90.70970328 -90.70970328 -90.70970328]
n:1272.0, a:0.9999441788794395


In [4]:
# done = False
# obs = env.reset()
# while not done:
#     ac = input()
#     ac = np.array([float(a) for a in ac])
#     obs, reward, done = env.step(ac)

In [5]:
obs = env.reset()

In [6]:
pg, h_coeffs = env.compute_channel_components()

In [7]:
env.decode_signal(np.array([0, 1, 1, 1]), pg, h_coeffs)

attempts: [1 2 3], eta: [3.64429670e-06 2.68336215e-06 2.45468411e-06 4.90973028e-06], decoding order: [3 1 2]
fast fading ||h||^2: [21.50720906 15.83615042 14.48658233 28.97530147]
Device: 3, sinr 198.57428443327942, interference: [1 2]
epsilon: 0.0, decoded: 1
Device: 1, sinr 290.9828492859938, interference: [2]
epsilon: 0.0, decoded: 1
Device: 2, sinr 5109.604015368565, interference: []
epsilon: 0.0, decoded: 1


(array([3, 1, 2]), [198.57428443327942, 290.9828492859938, 5109.604015368565])

### Test baselines

In [9]:
k = 8
deadlines = np.array([5 for _ in range(k)])
offsets = np.zeros(k)
arrival_probs = np.array([0.8 for _ in range(k)])
period = 5

env = NomaEnv(k,
              deadlines,
              lbda=1/9.3,
              period=period,
              arrival_probs=arrival_probs,
              offsets=offsets,
              episode_length=200,
              max_simultaneous_devices=3,
              traffic_model='aperiodic',
              channel_model='collision',
              distances=65,
              path_loss=True,
              shadowing=False,
              fast_fading=True,
              verbose=False
             )

In [10]:
random_scheduler = RandomScheduler(env)

In [11]:
random_scheduler.run(500)

0.8910196044859173

In [11]:
edf_scheduler = EarliestDeadlineFirstScheduler(env, True)

In [12]:
edf_scheduler.run(100)

Number of received packets: 40867.0


1.0

In [4]:
aloha = SlottedAloha(env, 0.5)

In [5]:
transmission_probs_list = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
cv = aloha.get_best_transmission_probs(50, transmission_probs_list)

In [6]:
cv

[0.36314019784610885,
 0.5743266363591925,
 0.6615205923631612,
 0.6845384087118611,
 0.5797185832120331,
 0.44992878542311276,
 0.2828614152718152,
 0.20614685245577624,
 0.12081692913385822,
 0.0827334630350195]

In [8]:
aloha.transmission_prob = transmission_probs_list[np.argmax(cv)]

In [12]:
aloha.run(500)

0.6706715686274509